In [2]:
! pip install SpeechRecognition

Looking in indexes: https://bytedpypi.byted.org/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 10.8 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 9.0 MB/s eta 0:00:00


In [4]:
! whereis pip
! pip list

pip: /Users/qinshuchang/Workspace/miniconda3/envs/sgooter/bin/pip
Package            Version
------------------ --------
appnope            0.1.4
asttokens          2.4.1
certifi            2024.2.2
cffi               1.16.0
charset-normalizer 3.3.2
comm               0.2.2
debugpy            1.8.1
decorator          5.1.1
exceptiongroup     1.2.0
executing          2.0.1
idna               3.7
importlib_metadata 7.1.0
ipykernel          6.29.3
ipython            8.22.2
jedi               0.19.1
jupyter_client     8.6.1
jupyter_core       5.7.2
matplotlib-inline  0.1.7
nest_asyncio       1.6.0
packaging          24.0
parso              0.8.4
pexpect            4.9.0
pickleshare        0.7.5
pip                24.0
platformdirs       4.2.1
pocketsphinx       5.0.3
prompt-toolkit     3.0.42
psutil             5.9.8
ptyprocess         0.7.0
pure-eval          0.2.2
PyAudio            0.2.14
pycparser          2.22
Pygments           2.17.2
python-dateutil    2.9.0
pyzmq              26.0.

In [3]:
import speech_recognition as sr

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import json



from sklearn.model_selection import train_test_split

samples = []
with open("intention.json", "r+") as file1:
    # Reading from a file
    samples = json.loads(file1.read())

# Assuming your data is stored in a list of dictionaries called 'data'
# 'data' should contain dictionaries with 'Intent' and 'Speech' keys

# Extract features (Speech) and labels (Intent) from the data
X = [sample['speech'] for sample in samples]
y = [sample['intent'] for sample in samples]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y)

# 定义一个SVM分类器
svm_classifier = Pipeline([
    ('tfidf', TfidfVectorizer()),  # 使用TF-IDF进行特征提取
    ('clf', SVC(kernel='linear'))  # 使用线性核的支持向量机
])

# 训练模型
svm_classifier.fit(X_train, y_train)

# 评估模型
y_pred = svm_classifier.predict(X_test)
print(y_pred)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy on test set:", accuracy)


['redirect' 'accelerate' 'stop' 'decelerate' 'stop' 'accelerate'
 'accelerate' 'decelerate' 'turn left' 'turn right' 'turn left'
 'turn left' 'decelerate' 'turn right' 'decelerate' 'accelerate'
 'turn right' 'redirect' 'redirect' 'stop' 'stop' 'stop' 'stop'
 'accelerate' 'accelerate' 'stop' 'turn right' 'turn right' 'stop'
 'turn left' 'turn left' 'decelerate' 'accelerate' 'stop' 'accelerate'
 'turn right']
Accuracy on test set: 0.9722222222222222


In [40]:
import pickle

# 保存模型
with open('intention.pkl', 'wb') as f:
    pickle.dump(svm_classifier, f)


In [51]:
import torch
import numpy as np
import wenet

model = wenet.load_model('english')

In [53]:
import speech_recognition as sr



# 创建一个Recognizer对象
recognizer = sr.Recognizer()

# 使用系统默认的麦克风
microphone = sr.Microphone()

# 开始录音
print("开始录音，请说话...")
with microphone as source:
    # 根据环境音量调整阈值（自动适应环境）
    recognizer.adjust_for_ambient_noise(source)
    # 监听麦克风输入，并将录制的音频存储到audio_data变量中
    try:
        audio_data = recognizer.listen(source, timeout=5)  # 设置超时时间为2秒
    except sr.WaitTimeoutError:
        print("录音超时，未能获取录音。")
        audio_data = None  # 将audio_data设为None，表示未获取到录音输入

print("录音结束")

# 如果获取到录音，则进行语音识别
if audio_data:
    try:
        text = recognizer.recognize_google(audio_data, language='en-US') # 识别英文
        print("识别结果：", text)
    except sr.UnknownValueError:
        print("无法识别音频内容")
    except sr.RequestError as e:
        print("请求音频识别失败：", e)
else:
    print("未获取到录音，不进行语音识别。")


开始录音，请说话...
录音结束
识别结果： left


In [2]:
import pickle
svm = None
with open('intention.pkl', 'rb') as f:
    svm = pickle.load(f)

In [3]:
import os
import time
import fcntl

file_lock_path = '/tmp/webots/intention.lock'
file_path = '/tmp/webots/intention'

def acquire_lock():
    lockfile = open(file_lock_path, "w")
    fcntl.flock(lockfile, fcntl.LOCK_EX)

def release_lock():
    lockfile = open(file_lock_path, "w")
    fcntl.flock(lockfile, fcntl.LOCK_UN)

def read_file():
    acquire_lock()
    with open(file_path, 'r+') as f:
        data = f.read()
        f.truncate(0) # reset
    release_lock()

    return data.strip()

def write_file(text):
    acquire_lock()
    with open(file_path, 'r+') as f:
        f.truncate(0) # reset
        f.write(text)
    release_lock()

In [4]:
import threading
import speech_recognition as sr

# 创建一个Recognizer对象
recognizer = sr.Recognizer()

# 使用系统默认的麦克风
microphone = sr.Microphone()

def listen_microphone():
    global recognizer, microphone
    
    while True:
        print("开始录音，请说话...")
        with microphone as source:
            # 根据环境音量调整阈值（自动适应环境）
            recognizer.adjust_for_ambient_noise(source)
            # 监听麦克风输入，并将录制的音频存储到audio_data变量中
            try:
                audio_data = recognizer.listen(source, timeout=5)  # 设置超时时间为2秒
            except sr.WaitTimeoutError:
                print("录音超时，未能获取录音。")
                audio_data = None  # 将audio_data设为None，表示未获取到录音输入
        print("录音结束")
        
        # 在另一个线程中处理语音识别结果
        threading.Thread(target=process_audio, args=(audio_data,), daemon=True).start()


command = None

def process_audio(audio_data):
    global recognizer
    
    if not audio_data:
        return

    try:
        print("GOOGLE>>>>>")
        text = recognizer.recognize_google(audio_data, language='en-US') # 识别英文
        print("识别结果：", text)
        
        intention = svm.predict([text])
        if len(intention)>0:
            command = intention[0]
            write_file(command)
        
    except sr.UnknownValueError:
        print("无法识别音频内容")
    except sr.RequestError as e:
        print("请求音频识别失败：", e)

# 启动监听麦克风的线程
threading.Thread(target=listen_microphone, daemon=True).start()

import time
while True:
    time.sleep(1)

    print("Cmd: ", command)
    command = None



开始录音，请说话...
Cmd:  None
Cmd:  None
Cmd:  None
Cmd:  None
录音结束
GOOGLE>>>>>
开始录音，请说话...
识别结果： accelerate
写入文件内容：accelerate
Cmd:  None
Cmd:  None
Cmd:  None
Cmd:  None
Cmd:  None
录音结束
GOOGLE>>>>>
开始录音，请说话...
无法识别音频内容
Cmd:  None
Cmd:  None
Cmd:  None
Cmd:  None
录音超时，未能获取录音。
录音结束
开始录音，请说话...
Cmd:  None
Cmd:  None
Cmd:  None
Cmd:  None
录音结束
GOOGLE>>>>>
开始录音，请说话...
Cmd:  None
识别结果： Pakistan rate
写入文件内容：accelerate
Cmd:  None
Cmd:  None
录音结束
GOOGLE>>>>>
开始录音，请说话...
无法识别音频内容
Cmd:  None
Cmd:  None
录音结束
GOOGLE>>>>>
开始录音，请说话...
无法识别音频内容
Cmd:  None
Cmd:  None
Cmd:  None
Cmd:  None
Cmd:  None
Cmd:  None
Cmd:  None
Cmd:  None
录音结束
GOOGLE>>>>>
开始录音，请说话...
无法识别音频内容
Cmd:  None
Cmd:  None
Cmd:  None
Cmd:  None
Cmd:  None
Cmd:  None
Cmd:  None
Cmd:  None
Cmd:  None
Cmd:  None
Cmd:  None
Cmd:  None
录音结束
GOOGLE>>>>>
开始录音，请说话...
Cmd:  None
识别结果： accelerate
写入文件内容：accelerate
Cmd:  None
Cmd:  None
Cmd:  None
Cmd:  None
Cmd:  None
Cmd:  None
Cmd:  None
Cmd:  None
Cmd:  None
Cmd:  None
Cmd:  None
Cmd:  None
Cmd:  